# Assignment2 

In [3]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from random import choice, sample, shuffle
import math
import os
import networkx as nx

from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

### 1. Load Training Data

In [4]:
with open("train.txt") as file :
    all_lines = file.read().splitlines()

followers_numb = Counter()
followers_list = defaultdict(list)
following_numb = Counter()
following_list = defaultdict(list)

all_nodes = []
source_nodes = []

for line in tqdm(all_lines) :
    nodes = line.split("\t")
    source, sinks = nodes[0], nodes[1:]
    sinks = list(map(int, sinks))
    source = int(source)
    while source in sinks: sinks.remove(source)
        
    following_list[source] = sinks
    following_numb[source] = len(sinks)
    
    for sink in sinks :
        followers_list[sink].append(source)
        followers_numb[sink] = len(followers_list[sink])
    
    all_nodes.append(source)
    all_nodes.extend(sinks)
    
all_nodes = list(set(all_nodes))

100%|██████████| 20000/20000 [00:32<00:00, 617.66it/s] 


Stats about `train.txt`

In [22]:
i = 0
null = 0

for node in following_numb :
    if following_numb[node] == 0 :
        null += 1
    else :
        i += following_numb[node]

print("Total lines in train.txt: ", len(all_lines))
print("Total edges in train.txt: ", i)
print("Total distinct nodes", len(all_nodes))
print("Size of Source nodes: ", len(edges))
print("Size of null entries: ", null)

Total lines in train.txt:  20000
Total edges in train.txt:  24004344
Total distinct nodes 4867136
Size of Source nodes:  20000
Size of null entries:  430


### 3. Loading Reduced Training Data

#### 3.1 Positive Data

In [5]:
pos_subPair = pd.read_csv('pos_subPair.csv', sep=",")
print(pos_subPair.head())
print(pos_subPair.shape)
pos_train_source_list = list(pos_subPair['source'])
pos_train_sink_list = list(pos_subPair['sink'])

print(len(pos_train_source_list), len(pos_train_sink_list))

   Unnamed: 0   source     sink
0           0   765601  2006214
1           1  1750842  4144037
2           2  1973082   229375
3           3  3361377  1542427
4           4  4230144  1246469
(20001, 3)
20001 20001


#### 3.2 Negative Data

In [6]:
neg_subPair = pd.read_csv('neg_subPair.csv', sep=",")
neg_subPair = neg_subPair.rename(columns = {'0': 'source', '1': 'sink'}, inplace = False)
print(neg_subPair.head())
print(neg_subPair.shape)
neg_train_source_list = list(neg_subPair['source'])
neg_train_sink_list = list(neg_subPair['sink'])

print(len(neg_train_source_list), len(neg_train_sink_list))

   Unnamed: 0   source     sink
0           0   493062     9330
1           1  4618437  2311748
2           2  4391648  1548304
3           3  1628298  1893139
4           4  1807730  4719681
(20001, 3)
20001 20001


### 4. Feature Engineering

#### 4.1 Common Friends

- **x1** - source → c → sink
- **x2** - source → c ← sink
- **x3** - source ← c → sink
- **x4** - source ← c ← sink
- **x5** - Jaccard's coeffcient

In [18]:
def gen_common_friends (source, sink, following_list, followers_list) :
    
    source_following, source_followers, sink_following, sink_followers = [], [], [], []
    x1, x2, x3, x4, x5 = 0, 0, 0, 0, 0
    
    if source in following_list :
        source_following = following_list[source]
        
    if source in followers_list :
        source_followers = followers_list[source]
    
    if sink in following_list :
        sink_following = following_list[sink]
        
    if sink in followers_list :
        sink_followers = followers_list[sink]
    
    if source_following != [] and sink_followers != [] :
        x1 = len(set(source_following).intersection(sink_followers))
        
    if source_following != [] and sink_following != [] :
        x2 = len(set(source_following).intersection(sink_following))
        
    if source_followers != [] and sink_followers != [] :
        x3 = len(set(source_followers).intersection(sink_followers))
    
    if source_followers != [] and sink_following != [] :
        x4 = len(set(source_followers).intersection(sink_following))
        
    related_source = set(source_following).union(source_followers)
    related_sink = set(sink_following).union(sink_followers)
    common_nodes = list(set(related_source).intersection(related_sink))
    related_both = len(set(related_source).union(related_sink))
#     if related_both != 0:
#         x5 = len(common_nodes)/related_both
    x5=len(common_nodes)
        
    return x1, x2, x3, x4, x5

In [14]:
test = gen_common_friends(765601, 2006214, following_list, followers_list)
print(test)
test = gen_common_friends(4391648, 1548304, following_list, followers_list)
print(test)

(59, 0, 57, 0, 0.0005899705014749262, 60)
(0, 0, 0, 0, 0.0, 0)


#### 4.1 Node stats

- **x6** - Number following (Source).
- **x7** - Number following (Sink).
- **x8** - Number of followers (Source).
- **x9** - Number of followers (Sink).

In [23]:
def gen_node_stats (source, sink, following_numb, followers_numb) :
    x6, x7, x8, x9 = 0, 0, 0, 0
    x6 = following_numb[source]
    x9 = followers_numb[sink]
    
    return x6, x9

#### 4.3 Generating Features

In [19]:
pos_train_features = []

for i in tqdm(range(len(pos_train_source_list))) :
    source = pos_train_source_list[i]
    sink = pos_train_sink_list[i]
    
    x1, x2, x3, x4, x5 = gen_common_friends(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    pos_train_features.append([x1, x2, x3, x4, x5, 1])
    
print(len(pos_train_features))

100%|██████████| 20001/20001 [09:47<00:00, 34.03it/s]

20001


In [27]:
pos_train_features2=[]
for i in tqdm(range(len(pos_train_source_list))) :
    source = pos_train_source_list[i]
    sink = pos_train_sink_list[i]
    x6, x9= gen_node_stats (source, sink, following_numb, followers_numb) 
    
    pos_train_features2.append([x6,x9])
    

100%|██████████| 20001/20001 [00:00<00:00, 400014.66it/s]


In [51]:
pos_train_features_total=[]


In [52]:
pos_train_features_total

[]

### 5. Training Model - First Stage

In [175]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90051)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))

Training set has 32001 instances. Test set has 8001 instances.


In [176]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='none')
model.fit(X, y)
w_sklearn = np.r_[model.intercept_, model.coef_.squeeze()]
print("Weights: {}".format(w_sklearn))

Weights: [ -1.19244575   0.51252288   0.14435657   0.29479233  -0.03321564
 -24.83857673]


### 6. Loading Test Data

In [7]:
testData = pd.read_csv('test-public.txt', sep="\t")
testData_ids = list(testData['Id'])
test_source_list = list(testData['Source'])
test_sink_list = list(testData['Sink'])
print(testData.head())
print(testData.shape)

   Id   Source     Sink
0   1  3563811  3600160
1   2  2052043  1401960
2   3  4517994  1690636
3   4  1660006  4349447
4   5   581111  1882617
(2000, 3)


In [77]:
test_features = []

for i in tqdm(range(len(test_source_list))) :
    source = test_source_list[i]
    sink = test_sink_list[i]
    
    x1, x2, x3, x4, x5, x6, x9 = gen_common_friends2(source, sink, following_list, followers_list,following_numb, followers_numb)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    test_features.append([x1, x2, x3, x4, x5,x6,x9])
    
print(len(test_features))

test_features = pd.DataFrame(test_features, columns = ['x1', 'x2', 'x3', 'x4', 'x5','x6','x7'])
print(test_features)
print(test_features.shape)
test_features.to_csv('test-features-new.csv', sep=",", index=False, header=True)

100%|██████████| 2000/2000 [00:01<00:00, 1344.08it/s]

2000
      x1  x2  x3  x4  x5   x6  x7
0      0   0   0   0   0   21  29
1      0   0   0   0   0   71   9
2      2   0   2   0   3  205  17
3      2   0   2   0   2  506  36
4      0   0   0   0   0   18  46
...   ..  ..  ..  ..  ..  ...  ..
1995   0   0   0   0   0   53   2
1996   0   0   1   0   1   95  41
1997   0   0   0   0   0   27   2
1998   0   0   0   0   0   56   3
1999   0   0   0   0   0  244   2

[2000 rows x 7 columns]
(2000, 7)


## Second Stage Feature: Adamic-Adar Feature

**x8** adamic-adar

In [12]:
def gen_adamic_adar (source, sink, following_list, followers_list) :
    
    source_following, source_followers, sink_following, sink_followers = [], [], [], []
    
    if source in following_list :
        source_following = following_list[source]
        
    if source in followers_list :
        source_followers = followers_list[source]
    
    if sink in following_list :
        sink_following = following_list[sink]
        
    if sink in followers_list :
        sink_followers = followers_list[sink]
    
    related_source = set(source_following).union(source_followers)
    related_sink = set(sink_following).union(sink_followers)
    common_nodes = list(set(related_source).intersection(related_sink))
#     if related_both != 0:
#         x5 = len(common_nodes)/related_both
    
    x6=0
    for common_node in common_nodes:
        num_follow_commonfriend=0
        if common_node in following_list:
            num_follow_commonfriend+=len(following_list[common_node])
        if common_node in followers_list:
            num_follow_commonfriend+=len(followers_list[common_node])
        if(num_follow_commonfriend!=0):
            x6+=1/math.log(num_follow_commonfriend)
        
    return x6

In [13]:
Admaic_Adar_features = []

for i in tqdm(range(len(pos_train_source_list))) :
    source = pos_train_source_list[i]
    sink = pos_train_sink_list[i]
    
    x8 = gen_adamic_adar(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features.append(x8)

100%|██████████| 20001/20001 [04:58<00:00, 67.11it/s] 


In [17]:
pd.DataFrame(Admaic_Adar_features, columns=['x8']).to_csv('Pos-Admaic-Adar-features.csv',index=False)

In [18]:
Admaic_Adar_features_neg = []

for i in tqdm(range(len(neg_train_source_list))) :
    source = neg_train_source_list[i]
    sink = neg_train_sink_list[i]
    
    x9 = gen_adamic_adar(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features_neg.append(x9)

100%|██████████| 20001/20001 [00:02<00:00, 7312.98it/s]


In [20]:
pd.DataFrame(Admaic_Adar_features_neg, columns=['x8']).to_csv('Neg-Admaic-Adar-features.csv',index=False)

In [22]:
pos_features= pd.read_csv('pos_feature_new.csv')

In [31]:
pos_jaccard = pd.read_csv('pos-features-common-friends-with-JC.csv')
pos_jaccard_feature = list(pos_jaccard['x5'])
pos_features.insert(9, 'x9',pos_jaccard_feature,True)

In [24]:
pos_features.insert(8, 'x8',Admaic_Adar_features,True)

In [26]:
neg_features= pd.read_csv('neg_feature_new.csv')
neg_features.insert(8, 'x8',Admaic_Adar_features_neg,True)

In [49]:
neg_jaccard = pd.read_csv('neg-features-common-friends-with-JC.csv')
neg_jaccard_feature = list(neg_jaccard['x5'])
neg_features.insert(9, 'x9',neg_jaccard_feature,True)

In [53]:
Admaic_Adar_features_test = []

for i in tqdm(range(len(test_source_list))) :
    source = test_source_list[i]
    sink = test_sink_list[i]
    
    x10 = gen_adamic_adar(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features_test.append(x10)
test_features = pd.read_csv('test-features-new.csv')
test_features.insert(7,'x8',Admaic_Adar_features_test,True)

100%|██████████| 2000/2000 [00:00<00:00, 2285.71it/s]


In [55]:
test_features = pd.read_csv('test-features-new.csv')
test_features.insert(7,'x8',Admaic_Adar_features_test,True)

In [57]:
test_jacc = pd.read_csv('test-features-common-friends-with-JC.csv')
test_jacc_feature=list(test_jacc['x5'])
test_features.insert(8,'x9',test_jacc_feature,True)

In [58]:
test_features

,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0,0,0,0,0,21,29,0.000000,0.000000
1,0,0,0,0,0,71,9,0.000000,0.000000
2,2,0,2,0,3,205,17,0.459422,0.011152
3,2,0,2,0,2,506,36,0.178376,0.003670
4,0,0,0,0,0,18,46,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1995,0,0,0,0,0,53,2,0.000000,0.000000
1996,0,0,1,0,1,95,41,0.100383,0.006061
1997,0,0,0,0,0,27,2,0.000000,0.000000
1998,0,0,0,0,0,56,3,0.000000,0.000000


## Third Stage: Features

- **x1** - source → c → sink
- **x2** - source → c ← sink
- **x3** - source ← c → sink
- **x4** - source ← c ← sink
- **x5** - total common friends
- **x6** - number of people source following
- **x7** - number of people followed sink
- **x8** - Adamic Adar
- **x9** - Jaccard Coefficient

In [34]:
pos_features.to_csv('Pos_Total_Feature.csv',index=False)

In [51]:
neg_features.to_csv('Neg_Total_Feature.csv',index=False)

In [59]:
test_features.to_csv('Test_Total_Feature.csv',index=False)

- **x10** - Hub Depressed
- **x11** - SI
- **x12** - SC
- **x13** - HP
- **x14** - PD
- **x15** - RA

In [9]:
def gen_common_friends3 (source, sink, following_list, followers_list) :
    
    source_following, source_followers, sink_following, sink_followers = [], [], [], []
    HD,HP,SI,SC,LHN,RA = 0, 0, 0, 0,0,0
    
    if source in following_list :
        source_following = following_list[source]
        
    if source in followers_list:
        source_followers = followers_list[source]
    
    if sink in following_list:
        sink_following = following_list[sink]
        
    if sink in followers_list:
        sink_followers = followers_list[sink]
        
    related_source = set(source_following).union(source_followers)
    related_sink = set(sink_following).union(sink_followers)
    common_nodes = list(set(related_source).intersection(related_sink))
    related_both = len(set(related_source).union(related_sink))
#     if related_both != 0:
#         x5 = len(common_nodes)/related_both
    HD = len(common_nodes)/ max(len(related_source),len(related_sink))
    HP = len(common_nodes)/ min(len(related_source),len(related_sink))
    SC = len(common_nodes)/ (math.sqrt(len(related_source)*len(related_sink)))
    SI = len(common_nodes)/(len(related_source)+len(related_sink))
    LHN = len(common_nodes)/(len(related_source)*len(related_sink))
    
        
    return HD,HP,SC,SI,LHN

In [10]:
HDlist=[]
HPlist=[]
SClist=[]
SIlist=[]
LHNlist=[]
for i in tqdm(range(len(pos_train_source_list))) :
    source = pos_train_source_list[i]
    sink = pos_train_sink_list[i]
    
    HD,HP,SC,SI,LHN = gen_common_friends3(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    HDlist.append(HD)
    HPlist.append(HP)
    SClist.append(SC)
    SIlist.append(SI)
    LHNlist.append(LHN)

100%|██████████| 20001/20001 [08:00<00:00, 41.64it/s]


In [55]:
pd.DataFrame(pos_additional_features).to_csv('pos_additional_features.csv')

In [33]:
pos_additional_features.insert(8,'LHN',LHNlist,True)

In [35]:
HDlistneg=[]
HPlistneg=[]
SClistneg=[]
SIlistneg=[]
LHNlistneg=[]
for i in tqdm(range(len(neg_train_source_list))) :
    source = neg_train_source_list[i]
    sink = neg_train_sink_list[i]
    
    HD,HP,SC,SI,LHN = gen_common_friends3(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    HDlistneg.append(HD)
    HPlistneg.append(HP)
    SClistneg.append(SC)
    SIlistneg.append(SI)
    LHNlistneg.append(LHN)

100%|██████████| 20001/20001 [00:04<00:00, 4705.01it/s]


In [36]:
neg_additional_features=pd.read_csv('Neg_Total_Feature.csv')

In [43]:
neg_additional_features=neg_additional_features.drop(columns=['HD','LHN','SI','SC','HP'])

In [44]:
neg_additional_features.insert(7,'HD',HDlistneg,True)
neg_additional_features.insert(7,'LHN',LHNlistneg,True)
neg_additional_features.insert(7,'SI',SIlistneg,True)
neg_additional_features.insert(7,'SC',SClistneg,True)
neg_additional_features.insert(7,'HP',HPlistneg,True)

In [56]:
pd.DataFrame(neg_additional_features).to_csv('neg_additional_features.csv')

In [47]:
HDlisttest=[]
HPlisttest=[]
SClisttest=[]
SIlisttest=[]
LHNlisttest=[]
for i in tqdm(range(len(test_source_list))) :
    source = test_source_list[i]
    sink = test_sink_list[i]
    
    HD,HP,SC,SI,LHN = gen_common_friends3(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    HDlisttest.append(HD)
    HPlisttest.append(HP)
    SClisttest.append(SC)
    SIlisttest.append(SI)
    LHNlisttest.append(LHN)

100%|██████████| 2000/2000 [00:01<00:00, 1615.51it/s]


In [48]:
test_additional_features=pd.read_csv('Test_Total_Feature.csv')

In [49]:
test_additional_features=test_additional_features.drop(columns=['x5','x6','x7'])

In [53]:
test_additional_features.insert(6,'HD',HDlisttest,True)
test_additional_features.insert(6,'LHN',LHNlisttest,True)
test_additional_features.insert(6,'SI',SIlisttest,True)
test_additional_features.insert(6,'SC',SClisttest,True)
test_additional_features.insert(6,'HP',HPlisttest,True)

In [57]:
pd.DataFrame(test_additional_features).to_csv('test_additional_features.csv')

## Fourth Stage Feature: Resource Allocation

In [8]:
def gen_resource_allocation (source, sink, following_list, followers_list) :
    
    source_following, source_followers, sink_following, sink_followers = [], [], [], []
    
    if source in following_list :
        source_following = following_list[source]
        
    if source in followers_list :
        source_followers = followers_list[source]
    
    if sink in following_list :
        sink_following = following_list[sink]
        
    if sink in followers_list :
        sink_followers = followers_list[sink]
    
    related_source = set(source_following).union(source_followers)
    related_sink = set(sink_following).union(sink_followers)
    common_nodes = list(set(related_source).intersection(related_sink))
#     if related_both != 0:
#         x5 = len(common_nodes)/related_both
    
    x6=0
    for common_node in common_nodes:
        num_follow_commonfriend=0
        if common_node in following_list:
            num_follow_commonfriend+=len(following_list[common_node])
        if common_node in followers_list:
            num_follow_commonfriend+=len(followers_list[common_node])
        if(num_follow_commonfriend!=0):
            x6+=1/(num_follow_commonfriend)
        
    return x6


In [9]:
Admaic_Adar_features = []

for i in tqdm(range(len(pos_train_source_list))) :
    source = pos_train_source_list[i]
    sink = pos_train_sink_list[i]
    
    x8 = gen_resource_allocation(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features.append(x8)

100%|██████████| 20001/20001 [05:03<00:00, 65.80it/s] 


In [10]:
Admaic_Adar_features_neg = []

for i in tqdm(range(len(neg_train_source_list))) :
    source = neg_train_source_list[i]
    sink = neg_train_sink_list[i]
    
    x9 = gen_resource_allocation(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features_neg.append(x9)

100%|██████████| 20001/20001 [00:02<00:00, 7033.87it/s]


In [11]:
pos_features= pd.read_csv('pos_additional_features.csv')
pos_features.insert(8, 'ra',Admaic_Adar_features,True)
neg_features= pd.read_csv('neg_additional_features.csv')
neg_features.insert(8, 'ra',Admaic_Adar_features_neg,True)

In [12]:
Admaic_Adar_features_test = []

for i in tqdm(range(len(test_source_list))) :
    source = test_source_list[i]
    sink = test_sink_list[i]
    
    x10 = gen_resource_allocation(source, sink, following_list, followers_list)
    #feat2 = gen_node_stats(source, sink, following_numb, followers_numb)
    
    Admaic_Adar_features_test.append(x10)
test_features = pd.read_csv('test_additional_features.csv')
test_features.insert(7,'ra',Admaic_Adar_features_test,True)

100%|██████████| 2000/2000 [00:00<00:00, 2463.06it/s]


In [16]:
pos_features.to_csv('pos_total_ra.csv')
neg_features.to_csv('neg_total_ra.csv')
test_features.to_csv('test_total_ra.csv')

# Final Selection Training Model

Final training model used features:
- **x1** - source → c → sink
- **x2** - source → c ← sink
- **x3** - source ← c → sink
- **x4** - source ← c ← sink
- **x8** - Adamic Adar
- **ra** - resource allocation

In [ ]:
pos_feature=pd.read_csv('pos_total_ra.csv')
neg_feature=pd.read_csv('neg_total_ra.csv')
train=pd.concat([pos_feature,neg_feature])
train1 = train.sample(frac=1).reset_index(drop=True)
x_test=pd.read_csv('test_total_ra.csv')
features = ['x1','x2','x3','x4','ra','x8']
x_train=train1[features].values
y_train=train1['y'].values
x_test=x_test[features].values

## 1. Logistic Regression

In [ ]:
model0 =LogisticRegression()
model0.fit(x_train, y_train)
pred = model0.predict_proba(x_test)[:, 1].tolist()

## 2. SVM 

In [ ]:
model1 = SVC(kernel='linear', probability=True)
preds1 = model1.fit(train_x,train_y)
prediction1 = model1.predict_proba(x_test)[:, 1].tolist()

In [ ]:
id = [i for i in range(1,2001)]
submission = pd.DataFrame({'Id':id, 'Predicted':prediction1})
